In [9]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd

In [10]:

def get_product_links():
    df = pd.DataFrame(columns=['product_name', 'product_review', 'user_rating'])
    
    base_url = 'https://www.beeradvocate.com'
    
    page = requests.get(base_url+'/beer/top-rated/')
    soup = BeautifulSoup(page.text,'html.parser')
    
    cells = soup.find_all('td')
    for cell in cells:
        if len(cell) == 2:            
            product = cell.find('b').text
            external_url = cell.find('a')['href']
            
            product_page = requests.get(base_url+external_url)
            tree = html.fromstring(product_page.content)
            for i in range(1,26):
                try:
                    rating = tree.xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[{}]/div[2]/span[1]/text()'.format(i))[0]
                    comments = tree.xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[{}]/div[2]/text()'.format(i))[1:-1]
                    comment = ""
                    for com in comments:
                        comment += com.strip('\n') + " "

                    df = df.append({'product_name': product,
                                    'product_review': comment, 
                                    'user_rating': rating}, 
                                    ignore_index=True)
                except:
                    pass

    return df 

df = get_product_links()

In [11]:
df.to_csv('reviews.csv',index=False)